<a href="https://colab.research.google.com/github/EktaU21/XAI_Cryptocurrency_Money_Laundering/blob/main/GraphLIME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-geometric

In [ ]:
! pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
! pip install torch-spline-conv -f https://data.pyg.org/whl/torch-{torch.__version__}.html

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch.nn import Linear, LayerNorm, ReLU, Dropout
import torch.nn.functional as F
from torch_geometric.nn import ChebConv, NNConv, DeepGCNLayer, GATConv, DenseGCNConv, GCNConv, GraphConv
from torch_geometric.data import Data, DataLoader

from sklearn.metrics import roc_auc_score
import scipy.sparse as sp

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import os
import networkx as nx
from tqdm.notebook import tqdm

#import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

#print(torch.__version__)
#!pip install torch-geometric

from torch_geometric.nn import GATConv
from torch_geometric.utils import to_networkx
#from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures

!pip install graphlime
from graphlime import GraphLIME

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
def test(srno):

  df_features = pd.read_csv('/content/drive/MyDrive/elliptic_bitcoin_dataset/elliptic_txs_features.csv', header=None)
  df_edges = pd.read_csv('/content/drive/MyDrive/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv')
  df_classes =  pd.read_csv('/content/drive/MyDrive/elliptic_bitcoin_dataset/elliptic_txs_classes.csv')
  df_classes['class'] = df_classes['class'].map({'unknown': 2, '1':1, '2':0})

  # merging dataframes
  df_merge = df_features.merge(df_classes, how='left', right_on="txId", left_on=0)
  df_merge = df_merge.loc[df_merge[1] == srno]
  df_merge = df_merge.sort_values(0).reset_index(drop=True)
  classified = df_merge.loc[df_merge['class'].loc[df_merge['class']!=2].index].drop('txId', axis=1)
  unclassified = df_merge.loc[df_merge['class'].loc[df_merge['class']==2].index].drop('txId', axis=1)

  # storing classified unclassified nodes seperatly for training and testing purpose
  classified_edges = df_edges.loc[df_edges['txId1'].isin(classified[0]) & df_edges['txId2'].isin(classified[0])]
  unclassified_edges = df_edges.loc[df_edges['txId1'].isin(unclassified[0]) | df_edges['txId2'].isin(unclassified[0])]
  del df_features, df_classes

  edgess =  df_edges.loc[df_edges['txId1'].isin(df_merge[0]) & df_edges['txId2'].isin(df_merge[0])]

  # all nodes in data
  nodes = df_merge[0].values
  map_id = {j:i for i,j in enumerate(nodes)} # mapping nodes to indexes

  edges = edgess.copy()
  edges.txId1 = edges.txId1.map(map_id)
  edges.txId2 = edges.txId2.map(map_id)

  edge_index = np.array(edges.values).T

  edge_index = torch.tensor(edge_index, dtype=torch.long).contiguous()
  weights = torch.tensor([1]* edge_index.shape[1] , dtype=torch.double)

  # maping txIds to corresponding indexes, to pass node features to the model
  node_features = df_merge.drop(['txId'], axis=1).copy()
  node_features[0] = node_features[0].map(map_id)
  classified_idx = node_features['class'].loc[node_features['class']!=2].index
  unclassified_idx = node_features['class'].loc[node_features['class']==2].index
  # replace unkown class with 0, to avoid having 3 classes, this data/labels never used in training
  node_features['class'] = node_features['class'].replace(2, 0)

  labels = node_features['class'].values
  node_features = torch.tensor(np.array(node_features.drop([0, 'class', 1], axis=1).values, dtype=np.double), dtype=torch.double)

  # converting data to PyGeometric graph data format
  data_train = Data(x=node_features, edge_index=edge_index, edge_attr=weights,
                               y=torch.tensor(labels, dtype=torch.double)) #, adj= torch.from_numpy(np.array(adj))

  y_train = labels[classified_idx]

  # spliting train set and validation set
  from sklearn.model_selection import train_test_split
  X_train, X_valid, y_train, y_valid, train_idx, valid_idx = train_test_split(node_features[classified_idx], y_train, classified_idx, test_size=0.15, random_state=42, stratify=y_train)

  train_idx
  ten_train_idx = torch.tensor(train_idx)
  ten_train_idx

  from torch_geometric.utils import index_to_mask, mask_to_index
  train_mask = index_to_mask(ten_train_idx, size= len(labels))
  train_mask.shape

  valid_idx
  ten_valid_idx = torch.tensor(valid_idx)
  ten_valid_idx

  from torch_geometric.utils import index_to_mask, mask_to_index
  val_mask = index_to_mask(ten_valid_idx, size= len(labels))
  val_mask.shape

  # converting data to PyGeometric graph data format
  data_train = Data(x=torch.tensor(node_features,dtype=torch.float),
                  edge_index=edge_index, #edge_attr=weights,
                               y=torch.tensor(labels, dtype=torch.long),
                  train_mask=torch.tensor(train_mask,dtype=torch.bool),
                  val_mask=torch.tensor(val_mask,dtype=torch.bool)) #, adj= torch.from_numpy(np.array(adj))

  print(data_train)
  #outt_file = '/content/drive/MyDrive/GraphLIME_TS/Information/result'+ str(srno) + '.txt'
  with open("/content/drive/MyDrive/GraphLIME_TS/Information/result.txt", "a+") as o:
    o.write('\nTime Step ' + str(srno) + '\n')
    o.write(f' {data_train}')

  #visualize the Graph
  plt.figure(figsize=(16, 12))
  G = to_networkx(data_train, to_undirected=True)

  #extract the degree value
  degrees = dict(nx.degree(G))
  node_indices = list(degrees.keys())
  node_degrees = list(degrees.values())

  max_degree = max(node_degrees)
  node_idx = np.argmax(node_degrees).item()

  plt.figure(figsize=(16, 4))
  plt.bar(node_indices, node_degrees, width=5.0)
  plt.vlines(x=node_idx, ymin=0, ymax=max_degree, colors='r')
  plt.title('Degree of Nodes for Time Step'+str(srno))
  plt.xlabel('Node Index')
  plt.ylabel('Degree');

  rootdir = '/content/drive/MyDrive/GraphLIME_TS/Node_degree/'
  plt.rcParams["savefig.directory"] = os.chdir(os.path.dirname(rootdir))
  out_file = 'Node_TS'+ str(srno) + '.png'
  plt.savefig(out_file)
  plt.close()

  with open("/content/drive/MyDrive/GraphLIME_TS/Information/result.txt", "a+") as o:
    o.write("\n")
    o.write(f'Node {node_idx} has the largest degree value {max_degree}.')

  #define a GAT class

  class GAT(nn.Module):

    def __init__(self, input_dim, hidden_dim, output_dim,
                 heads_1=8, heads_2=1, att_dropout=0.6, input_dropout=0.6):

        super(GAT, self).__init__()

        self.att_dropout = att_dropout
        self.input_dropout = input_dropout

        self.conv1 = GATConv(in_channels=input_dim,
                             out_channels=hidden_dim // heads_1,
                             heads=heads_1,
                             concat=True,
                             dropout=att_dropout)
        self.conv2 = GATConv(in_channels=hidden_dim,
                             out_channels=output_dim,
                             heads=heads_2,
                             concat=False,
                             dropout=att_dropout)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=self.input_dropout, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=self.input_dropout, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

 #Instantiate a GAT model
  hparams = {
    'input_dim': data_train.num_node_features,
    'hidden_dim':8,
    'output_dim': int(max(data_train.y).item()) + 1
  }
  model = GAT(**hparams)
  model

  #Train the model
  def accuracy(output, labels):
    _, pred = output.max(dim=1)
    correct = pred.eq(labels).double()
    correct = correct.sum()

    return correct / len(labels)

  lr = 0.005
  #change the lr and epochs to 0.01 and 200
  epochs =300

  model.train()
  optimizer = optim.Adam(model.parameters(), lr=lr)

  for epoch in tqdm(range(epochs)):
    optimizer.zero_grad()

    output = model(data_train.x, data_train.edge_index)
    loss = F.nll_loss(output[data_train.train_mask], data_train.y[data_train.train_mask])

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        acc = accuracy(output[data_train.train_mask], data_train.y[data_train.train_mask])
        #print('Epoch: {:3d}, acc = {:.3f}'.format(epoch, acc))
        with open("/content/drive/MyDrive/GraphLIME_TS/Information/result.txt", "a+") as o:
          o.write("\n")
          o.writelines('Epoch: {:3d}, acc = {:.3f}'.format(epoch, acc))
          o.write("\n")

  #Explain node features
  node_idx
  model.eval()

  #Compute the coefficients of features

  # instantiate a GraphLIME object
  explainer = GraphLIME(model, hop=2, rho=0.1, cached=True)

  # explain node features by calling the method `explain_node()`
  coefs = explainer.explain_node(node_idx, data_train.x, data_train.edge_index)
  coefs

  with open("/content/drive/MyDrive/GraphLIME_TS/Information/matrix.csv", "a+") as o:
    o.write("Coefficient of Time Step"+str(srno)+'\n')
    o.write(f' {coefs}')
    o.write("\n")

  #visualize the coefficeints

  plt.figure(figsize=(16, 4))

  x = list(range(data_train.num_node_features))

  plt.bar(x, coefs, width=5.0)
  plt.xlabel('Feature Index')
  plt.ylabel(r'$\beta$');
  plt.title("Time Step"+str(srno));

  rootdir = '/content/drive/MyDrive/GraphLIME_TS/features/'
  plt.rcParams["savefig.directory"] = os.chdir(os.path.dirname(rootdir))
  output_file = 'TS'+ str(srno) + '.png'
  plt.savefig(output_file)
  plt.close()

  print(f'The {np.argmax(coefs)}-th feature is the most important.')
  with open("/content/drive/MyDrive/GraphLIME_TS/Information/result.txt", "a+") as o:
          o.write("\n")
          o.write(f'The {np.argmax(coefs)}-th feature is the most important.')

#test() is called by passing the desired time step number as an argument.
for i in range(1,50):
  test(i)